# Mix

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage 

import os
import cv2

import pandas as pd
%matplotlib inline

In [2]:
# Load images and resize 
root= './train'

def load_data(root_path):
    names = []
    train_crude = []

    for file in os.listdir(root):
        img = cv2.imread(os.path.join(root, file))#,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img.shape[1]//3,img.shape[0]//3))
        train_crude.append(img)
        names.append(file.split('.')[0])
    
    data = np.stack( train_crude, axis=0 )
    return names, data

names, data = load_data(root)

In [3]:
data.shape

(7500, 163, 226, 3)

## Extract feature CNN

In [4]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

base_model = ResNet50V2(weights='imagenet',input_shape=[163, 226,3],include_top=False)

In [5]:
#get features in batches and flatten

def generate_features(data,base_model, test = False):
    feat = []
    batch_size = 500
    num_samples = data.shape[0]
    if test:
        batch_size = 10
    for i in range(0,num_samples,batch_size):
        temp = base_model.predict(preprocess_input(data[i:i + batch_size]))
        feat.append(temp)


    feat0 = np.concatenate(feat, axis = 0) 
    feat2 = [i.ravel() for i in feat0]
    feat2 = np.stack(feat2, axis = 0) #these are the features
    
    return feat2

feat = generate_features(data,base_model)

In [6]:
print(feat.shape)

(7500, 98304)


## Nearest neighbor

In [7]:

from sklearn.neighbors import NearestNeighbors

def get_nearest_neighbours(features, *args):
    
    neighTest = NearestNeighbors(n_neighbors = 5)
    predictions = None
    
    if not args:
        train = features[:7000]
        validation = features[7000:]
        neighTest.fit(train)
        predictions = neighTest.kneighbors(validation)
    else:
        test_features = args[0] 
        neighTest.fit(features)
        predictions = neighTest.kneighbors(test_features)
    
    return predictions
        
predictions = get_nearest_neighbours(feat)

In [8]:
predictions[0][0] # Distances

array([437.64816, 450.4767 , 455.55823, 458.5046 , 458.75742],
      dtype=float32)

In [9]:
predictions[1][0] # Indices

array([2964,  684,  322, 6431, 2433], dtype=int64)

## SIFT extraction for neighbor

In [10]:
#Generates de key points and descriptors of the neighbours of the images to be predicted

def generate_descriptors_neighbours(predictions):
    sift = cv2.SIFT_create()

    key_points = []
    descriptors = []
    for  i, i_n in enumerate(predictions[1]): #Iterate over indices of the predicted nearest neighbours
        # print(i_n)
        kp_neighbours = []
        desc_neighbours = []
        for neigh in i_n: # Iterates over the indices of the k nearest neighbours of the instance being predicted
            img = data[neigh]
            gray_neighbour = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)# A neighbour of the instances being predicted
            kp, des = sift.detectAndCompute(gray_neighbour,None) # key points and descriptors of the neighbour of instance being predicted
            kp_neighbours.append(kp)
            desc_neighbours.append(des)

        key_points.append(kp_neighbours)
        descriptors.append(desc_neighbours)
    
    return key_points, descriptors

kpts_target_cand,descriptors_cand = generate_descriptors_neighbours(predictions)


In [11]:
len(descriptors_cand[0])
descriptors_cand[0]

[array([[  5.,   1.,  15., ...,   2.,  27.,  20.],
        [  1.,  10.,  14., ...,   1.,   0.,   0.],
        [  4.,   9.,   7., ...,  12.,   0.,   0.],
        ...,
        [ 14.,   2.,   1., ...,   3.,   9.,   3.],
        [  1.,  49.,  35., ...,   0.,   0.,   0.],
        [ 86., 108.,   1., ...,   0.,   0.,   0.]], dtype=float32),
 array([[ 0.,  1., 62., ...,  2.,  0.,  2.],
        [ 0.,  0., 61., ..., 12.,  2.,  9.],
        [39.,  1.,  0., ...,  1.,  3.,  1.],
        ...,
        [ 8.,  0.,  0., ..., 42., 15.,  1.],
        [ 2.,  0.,  0., ...,  5.,  2.,  4.],
        [ 0.,  5., 76., ...,  0.,  0.,  0.]], dtype=float32),
 None,
 None,
 None]

In [12]:
#Generates de key points and descriptors of the images to be predicted
validation_img = data[7000:]

def generate_descriptors(data_img):
    sift = cv2.SIFT_create()
    
    key_points = []
    descript = []

    for  img in data_img: #Iterate over indices of predictions
        # print(i_n)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray,None) # key points and descriptors of the instance being predicted
        key_points.append(kp)
        descript.append(des)
    
    return key_points, descript

kpts_target,descriptors_target = generate_descriptors(validation_img)
#descript_dev=np.array(descript_dev)

In [13]:
len(descriptors_target)
len(descriptors_target[0][0])

128

In [14]:
class PtsWrapper():
    def __init__(self, data):
        self.puntos = data
    
    def get_points():
        return self.puntos
        

In [15]:
def find_matches(matcher,kp1, des1, kp2, des2):
    # Matching descriptor using KNN algorithm
    try:
        matches = matcher.knnMatch(des1, des2, k = 2)
    except:
        #print('exception in find matches')
        return None, None, None

    # Store all good matches as per Lowe's Ratio test.
    ratio = 0.6
    num_matches = 0
    ptsL = []
    ptsR = []
    for m,n in matches:
        if m.distance < ratio * n.distance:
            num_matches += 1
            ptsL.append(kp1[m.queryIdx].pt)
            ptsR.append(kp2[m.trainIdx].pt)
            
    ptsL = np.int32(ptsL)
    ptsR = np.int32(ptsR)
    
    return num_matches, ptsL, ptsR

def match(kpts_target ,desc_targets, kpts_candidates, desc_candidates, predictions):
    """Finds the neighbours that best match the target image based on their descriptors

    Keyword arguments:
    kpts_target -- list of key points from the target images 
    desc_targets -- list of descriptors from the target images 
    kpts_candidates -- list (of lists) of key points from the nearest neighbours of the target images
    desc_candidates -- list (of lists) of descriptors from the nearest neighbours of the target images
    predictions = tuple containing two matrices. The first one with the distance from the nearest neighbours to the target images. 
                                                The second one with the indices of the nearest neighbours from the target images.
    Returns a matrix of matrices. Dim 0 corresponds to a matrix containing information relative to the neighbours of the target image.
                                The matrix is of shape (number of neighbours matched ,3) where the first column is the index of the neighbour in the coordinates file,
                                Second column indicates the number of matches of that neighbour with the target image
                                Third column is an object of class ptsWrapper that contains the list of matched  key points of that neighbour with the target image (use get_points() to access them) 
    """
    
    # FLANN parameters and initialize
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    best_neighbours_matched = []
    for i, target in enumerate(desc_targets):
        # print('\n\n')
        # print('image {}'.format(i))
        matches_list = []
        ind_neighbour = []
        left_pts_list = []
        right_pts_list = []
        for j, candidate in enumerate(desc_candidates[i]):
            #print(j)
            if candidate is not None:
                #print('neighb {}'.format(j))
                n_matches, pts_L, pts_R = find_matches(flann, kpts_target[i], target,  kpts_candidates[i][j], candidate)
                if n_matches is not None:
                    ind_neighbour.append(predictions[1][i][j]) #Appends index in the data of the neighbour from the image being predicted
                    matches_list.append(n_matches) #Appends matches between the current neighbour and the instance being predicted
                    left_pts_list.append(PtsWrapper(pts_L))
                    right_pts_list.append(PtsWrapper(pts_R))
                else:
                    continue
        
        #print('len ind neigh',ind_neighbour)
        #print('len matches list',matches_list)
        neighb_matches = np.stack([ind_neighbour, matches_list, left_pts_list, right_pts_list], axis = 1) #Creates numpy array. Rows are descriptors of different neighbours. 
                                                                     # First column is index in the data, Second column is how many matches that neighbour has with the current instance
        neighb_matches = neighb_matches[neighb_matches[:, 1].argsort()[::-1][:len(matches_list)]]  #Sorts array in descending order, more matches on top and less matches at the bottom :5 is the number of neighbours
        #print(neighb_matches)
        best_neighbours_matched.append(neighb_matches)
    
    return best_neighbours_matched
                                 


In [16]:
k_best_matches = match(kpts_target, descriptors_target, kpts_target_cand, descriptors_cand, predictions)

In [26]:
## See output example below to understand the output of the function match()
exmp = k_best_matches[0][0][2]
exmp.puntos


array([[98, 78],
       [98, 78]])

In [25]:
exmp

## Get Essential matrix

In [34]:
def camera_matrix(horizontal, vertical, img):
    x = img.shape[0] / 2
    y = img.shape[1] / 2
    f_x = (x / np.tan(horizontal * (np.pi/180)/2)) 
    f_y = (y / np.tan(vertical * (np.pi/180)/2))
    return np.array([[f_x,0,x],
                     [0,f_y,y],
                     [0, 0, 1]])
    

In [36]:
F, mask = cv2.findFundamentalMat(k_best_matches[0][0][2].puntos,k_best_matches[0][0][3].puntos,cv2.FM_LMEDS)

In [38]:
K = camera_matrix(73.7, 53.1, data[0])

In [ ]:
E = K.t*F*K

In [41]:
K.T

array([[108.7453233 ,   0.        ,   0.        ],
       [  0.        , 226.14849934,   0.        ],
       [ 81.5       , 113.        ,   1.        ]])

In [35]:
camera_matrix(57.5, 45, np.zeros((640,480)))

array([[583.28297864,   0.        , 320.        ],
       [  0.        , 579.41125497, 240.        ],
       [  0.        ,   0.        ,   1.        ]])

## TEST predictions

In [63]:
root= './test'
names_test, test_img = load_data(root)


In [64]:
test_feat=generate_features(test_img,base_model, test=True)

In [68]:
test_feat.shape

(1200, 98304)

In [69]:
predicted_neigh = get_nearest_neighbours(feat, test_feat)

In [70]:
kps_neigh, descript_neigh = generate_descriptors_neighbours(predicted_neigh)

In [71]:
kps_target, descript_target=generate_descriptors(test_img)

In [ ]:
k_best_matches_test=match(kps_target, descript_target, kps_neigh, descript_neigh,predicted_neigh)

In [119]:


def give_preds(k_best_matches_test,predicted_neigh, df_coordinates,names_test):
    """Peforms the prediction for the test_set using the k best matches or the best neighbours in case there are no matches produced by SIFT

    Keyword arguments:
    k_best_matches_test -- k best matches list generated by feature matching using SIFT
    predicted_neigh -- predicted k nearest neighbours by the pre-trained model
    df_coordinates -- dataframe where the coordinates of the training points are
    names_test =
    """
    pred_coordinates=[]
    record_no_matches=[]
    for i, best in enumerate(k_best_matches_test):
        if best.size != 0:
            index_best=best[0][0] # This is the index of the neighbour with more matches
            pred_coordinates.append([names_test[i],df_coordinates.loc[index_best,'x'],df_coordinates.loc[index_best,'y']])
        else:
            # if there were no matches, use the average of the locations of the neighbours found by the pre-trained model
            num_neighbors=2
            x_coord_arr=np.array([df_coordinates.loc[j,'x'] for j in predicted_neigh[1][i]])
            x_coord=x_coord_arr[:num_neighbors].mean()
            y_coord_arr=np.array([df_coordinates.loc[j,'y'] for j in predicted_neigh[1][i]])
            y_coord=y_coord_arr[:num_neighbors].mean()
            pred_coordinates.append([names_test[i], x_coord, y_coord])
    
    array_preds=np.stack(pred_coordinates, axis=0)
   
    df_predictions=pd.DataFrame(array_preds, columns=['id','x','y'])
    return df_predictions

#Load the coordinates data set
coordinates = pd.read_csv('train.csv')
            
df_preds = give_preds(k_best_matches_test, predicted_neigh, coordinates,names_test)
        
    
    

In [120]:
df_preds.to_csv('resnet_knn_sift2.csv',index=False)

In [117]:
predicted_neigh

def transform predicted(predictions):

(array([[323.00092, 396.38666, 405.2545 , 406.16324, 406.93503],
        [402.14273, 403.9036 , 405.39566, 410.11633, 415.00497],
        [404.2133 , 407.3134 , 407.66003, 410.6764 , 412.31476],
        ...,
        [523.98206, 528.6266 , 529.6274 , 531.1956 , 531.5879 ],
        [431.3342 , 439.16223, 439.67752, 441.6945 , 441.93524],
        [375.47748, 380.55112, 382.6583 , 385.34314, 385.38266]],
       dtype=float32),
 array([[3158, 7050, 5008, 6206,  290],
        [3338, 1605, 6744, 7364, 2334],
        [2334, 4004, 6860, 3020, 1961],
        ...,
        [2334, 3020, 5285, 6414, 1961],
        [2334, 2433, 4543, 4008, 6949],
        [2334, 2433,  322, 2849, 3809]], dtype=int64))